In [19]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [35]:
import pandas as pd
import numpy as np
import graphviz
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import scipy.stats as stats
import warnings
import re

from statsmodels.formula.api import ols
from sklearn.feature_selection import RFE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.tree import export_graphviz
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from statsmodels.api import qqplot, add_constant
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
warnings.filterwarnings('ignore')

In [37]:
# 로지스틱 회귀
from statsmodels.api import Logit
# 분류모델 평가 함수
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# export_graphviz: 나무 구조 생성 및 저장
from sklearn.tree import export_graphviz
# graphviz : 나무 구조 시각화  (.dot 확장자 파일 불러오기 등)
import graphviz
# 다른 방식(.dot -> .png 형식, 출력화면에 맞는)으로 Tree 출력
from subprocess import call
from IPython.display import Image
# 분류 Decision Tree
from sklearn.tree import DecisionTreeClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# 분류 Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# xgboost
from xgboost import XGBClassifier
# randomforest
from sklearn.ensemble import RandomForestClassifier
# lightgbm
from lightgbm import LGBMClassifier
# svm
from sklearn.svm import SVC
# mlp
from sklearn.neural_network import MLPClassifier

**밑 코드 실행하고 세션 다시 시작 후 실행 필요**

In [3]:
#코랩 코드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (24.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126209 files and direc

In [21]:
import matplotlib as mpl
import matplotlib.pyplot as plt
# 유니코드 깨짐현상 해결
mpl.rcParams['axes.unicode_minus'] = False

# 나눔고딕 폰트 적용
plt.rcParams["font.family"] = 'NanumGothic'

In [22]:
# 실행결과 경고메시지 출력 제외
import warnings
import platform
warnings.filterwarnings('ignore')

In [38]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# ConvergenceWarning 숨기기
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# 데이터 불러오기

In [23]:
df_raw = pd.read_csv("/content/drive/MyDrive/포스코 빅데이터 아카데미/빅데이터 프로젝트/후판 scale 불량 예측 프로젝트/SCALE불량_추가.csv", encoding = 'euc-kr')
df_raw.head()

,plate_no,rolling_date,scale,spec_long,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,...,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,항복강도(MPa),용도
0,PLT_1613,07JAN2023:14:07:43,불량,A131-DH36TM,미국,T,71,3600,9200,미적용,...,92,1121,145,332,CR(제어압연),859,6,3조,355.0,선체구조용
1,PLT_1798,08JAN2023:21:04:46,양품,A283-C,미국,C,18,2300,40800,적용,...,73,1155,86,237,CR(제어압연),981,10,4조,245.0,용접구조용
2,PLT_1818,08JAN2023:23:54:27,불량,A283-C,미국,C,26,2600,31300,적용,...,73,1160,63,223,CR(제어압연),1026,8,4조,245.0,용접구조용
3,PLT_1819,09JAN2023:00:04:11,불량,A283-C,미국,C,26,2600,31300,적용,...,69,1177,65,230,CR(제어압연),1036,8,4조,245.0,용접구조용
4,PLT_1891,09JAN2023:11:39:50,불량,A283-C,미국,C,26,2500,30000,적용,...,61,1164,57,210,CR(제어압연),1034,8,3조,245.0,용접구조용


# 통계 분석

In [24]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   plate_no         1000 non-null   object 
 1   rolling_date     1000 non-null   object 
 2   scale            1000 non-null   object 
 3   spec_long        1000 non-null   object 
 4   spec_country     1000 non-null   object 
 5   steel_kind       1000 non-null   object 
 6   pt_thick         1000 non-null   int64  
 7   pt_width         1000 non-null   int64  
 8   pt_length        1000 non-null   int64  
 9   hsb              1000 non-null   object 
 10  fur_no           1000 non-null   object 
 11  fur_input_row    1000 non-null   object 
 12  fur_heat_temp    1000 non-null   int64  
 13  fur_heat_time    1000 non-null   int64  
 14  fur_soak_temp    1000 non-null   int64  
 15  fur_soak_time    1000 non-null   int64  
 16  fur_total_time   1000 non-null   int64  
 17  rolling_method 

In [25]:
df_raw.describe()

,pt_thick,pt_width,pt_length,fur_heat_temp,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_temp,descaling_count,항복강도(MPa)
count,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,964.000000
mean,26.78200,2831.900000,36788.200000,1157.245000,85.972000,1150.928000,71.720000,238.589000,934.637000,8.557000,292.686722
std,18.13757,494.081478,13912.387116,21.245007,26.346297,17.344384,20.602137,38.194828,96.598015,1.604158,52.128539
min,12.00000,1800.000000,7900.000000,1103.000000,55.000000,1113.000000,35.000000,165.000000,0.000000,5.000000,205.000000
25%,15.00000,2500.000000,26650.000000,1140.000000,66.000000,1135.750000,57.750000,210.000000,893.750000,8.000000,235.000000
50%,19.00000,2800.000000,40400.000000,1159.000000,75.000000,1156.000000,66.000000,230.000000,948.000000,9.000000,315.000000
75%,34.00000,3100.000000,49100.000000,1173.000000,102.250000,1164.000000,81.000000,263.000000,991.000000,10.000000,325.000000
max,100.00000,4600.000000,54900.000000,1206.000000,158.000000,1185.000000,145.000000,362.000000,1078.000000,10.000000,600.000000


# 결측치 처리

In [26]:
df_raw.isnull().sum()

# 결측치 존재하지 않음

,0
plate_no,0
rolling_date,0
scale,0
spec_long,0
spec_country,0
steel_kind,0
pt_thick,0
pt_width,0
pt_length,0
hsb,0


# 데이터 전처리

In [27]:
# plate_no 는 분석에 필요하지 Plate 번호 정보 → 열 삭제
df = df_raw.drop(labels = 'plate_no', axis = 1)
df.head()

,rolling_date,scale,spec_long,spec_country,steel_kind,pt_thick,pt_width,pt_length,hsb,fur_no,...,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,항복강도(MPa),용도
0,07JAN2023:14:07:43,불량,A131-DH36TM,미국,T,71,3600,9200,미적용,2호기,...,92,1121,145,332,CR(제어압연),859,6,3조,355.0,선체구조용
1,08JAN2023:21:04:46,양품,A283-C,미국,C,18,2300,40800,적용,2호기,...,73,1155,86,237,CR(제어압연),981,10,4조,245.0,용접구조용
2,08JAN2023:23:54:27,불량,A283-C,미국,C,26,2600,31300,적용,2호기,...,73,1160,63,223,CR(제어압연),1026,8,4조,245.0,용접구조용
3,09JAN2023:00:04:11,불량,A283-C,미국,C,26,2600,31300,적용,2호기,...,69,1177,65,230,CR(제어압연),1036,8,4조,245.0,용접구조용
4,09JAN2023:11:39:50,불량,A283-C,미국,C,26,2500,30000,적용,3호기,...,61,1164,57,210,CR(제어압연),1034,8,3조,245.0,용접구조용


spec_long 변수 split

In [28]:
# 분할 함수 정의
def split_spec(val):
    if '/' in val or '-' in val:
        parts = [p for p in re.split('[-/]', val) if p]
        first = parts[0] if len(parts) >= 1 else ''
        second = parts[1] if len(parts) >= 2 else ''
        third = parts[2] if len(parts) >= 3 else ''
    else:
        first, second, third = val, '', ''
    return pd.Series([first, second, third], index=['spec_long(1)', 'spec_long(2)', 'spec_long(3)'])

# spec_long 열 오른쪽 위치 찾기
col_index = df.columns.get_loc('spec_long') + 1

# 새로운 열 생성
new_cols = df['spec_long'].apply(split_spec)

# 열을 해당 위치에 삽입
for i, col_name in enumerate(new_cols.columns):
    df.insert(col_index + i, col_name, new_cols[col_name])

# 결과 확인
df.head()

,rolling_date,scale,spec_long,spec_long(1),spec_long(2),spec_long(3),spec_country,steel_kind,pt_thick,pt_width,...,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,항복강도(MPa),용도
0,07JAN2023:14:07:43,불량,A131-DH36TM,A131,DH36TM,,미국,T,71,3600,...,92,1121,145,332,CR(제어압연),859,6,3조,355.0,선체구조용
1,08JAN2023:21:04:46,양품,A283-C,A283,C,,미국,C,18,2300,...,73,1155,86,237,CR(제어압연),981,10,4조,245.0,용접구조용
2,08JAN2023:23:54:27,불량,A283-C,A283,C,,미국,C,26,2600,...,73,1160,63,223,CR(제어압연),1026,8,4조,245.0,용접구조용
3,09JAN2023:00:04:11,불량,A283-C,A283,C,,미국,C,26,2600,...,69,1177,65,230,CR(제어압연),1036,8,4조,245.0,용접구조용
4,09JAN2023:11:39:50,불량,A283-C,A283,C,,미국,C,26,2500,...,61,1164,57,210,CR(제어압연),1034,8,3조,245.0,용접구조용


rolling_date 변수 split

In [29]:
# 날짜 분리 함수 정의
def split_rolling_date(date_str):
    match = re.match(r"(\d{2})([A-Z]{3})(\d{4}):(\d{2}:\d{2}:\d{2})", date_str)
    if match:
        day, month, year, time = match.groups()
        return pd.Series([year, month, day, time], index=['year', 'month', 'day', 'time'])
    else:
        return pd.Series(['', '', '', ''], index=['year', 'month', 'day', 'time'])

# rolling_date 열 위치 찾기
col_index = df.columns.get_loc('rolling_date') + 1

# 새 열 생성
new_cols = df['rolling_date'].apply(split_rolling_date)

# 원래 데이터프레임에 열 삽입
for i, col_name in enumerate(new_cols.columns):
    df.insert(col_index + i, col_name, new_cols[col_name])

# 결과 확인
df.head()

,rolling_date,year,month,day,time,scale,spec_long,spec_long(1),spec_long(2),spec_long(3),...,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,항복강도(MPa),용도
0,07JAN2023:14:07:43,2023,JAN,07,14:07:43,불량,A131-DH36TM,A131,DH36TM,,...,92,1121,145,332,CR(제어압연),859,6,3조,355.0,선체구조용
1,08JAN2023:21:04:46,2023,JAN,08,21:04:46,양품,A283-C,A283,C,,...,73,1155,86,237,CR(제어압연),981,10,4조,245.0,용접구조용
2,08JAN2023:23:54:27,2023,JAN,08,23:54:27,불량,A283-C,A283,C,,...,73,1160,63,223,CR(제어압연),1026,8,4조,245.0,용접구조용
3,09JAN2023:00:04:11,2023,JAN,09,00:04:11,불량,A283-C,A283,C,,...,69,1177,65,230,CR(제어압연),1036,8,4조,245.0,용접구조용
4,09JAN2023:11:39:50,2023,JAN,09,11:39:50,불량,A283-C,A283,C,,...,61,1164,57,210,CR(제어압연),1034,8,3조,245.0,용접구조용


In [ ]:
df.to_excel("/content/drive/MyDrive/포스코 빅데이터 아카데미/빅데이터 프로젝트/후판 scale 불량 예측 프로젝트/scale.csv", index=False)

# 이상치 처리

In [30]:
sns.pairplot(df)

Output hidden; open in https://colab.research.google.com to view.

그래프 확인 결과, rolling_temp가 0인 이상치를 확인할 수 있다

In [31]:
outliers = df[df['rolling_temp'] == 0]
outliers

# rolling_temp가 0인 이상치는 총 6개

,rolling_date,year,month,day,time,scale,spec_long,spec_long(1),spec_long(2),spec_long(3),...,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,항복강도(MPa),용도
395,03JAN2023:16:22:29,2023,JAN,03,16:22:29,양품,GL-E36-TM,GL,E36,TM,...,95,1128,80,244,TMCP(온도제어),0,6,1조,355.0,선체구조용
497,04JAN2023:19:44:09,2023,JAN,04,19:44:09,양품,JS-SM490YB,JS,SM490YB,,...,57,1166,67,228,CR(제어압연),0,10,2조,355.0,용접구조용
498,04JAN2023:19:56:39,2023,JAN,04,19:56:39,양품,JS-SM490YB,JS,SM490YB,,...,64,1164,61,188,CR(제어압연),0,10,2조,355.0,용접구조용
922,07JAN2023:11:59:10,2023,JAN,07,11:59:10,양품,NV-D32-TM,NV,D32,TM,...,109,1117,76,260,TMCP(온도제어),0,6,3조,315.0,선체구조용
973,07JAN2023:11:40:51,2023,JAN,07,11:40:51,양품,PILAC-BT33,PILAC,BT33,,...,89,1122,59,229,CR(제어압연),0,6,3조,325.0,일반구조용
974,07JAN2023:11:52:00,2023,JAN,07,11:52:00,양품,PILAC-BT33,PILAC,BT33,,...,92,1124,61,210,CR(제어압연),0,6,3조,325.0,일반구조용


In [32]:
group_cols = ['spec_long', 'spec_country', 'steel_kind', 'rolling_method']
df['rolling_temp'] = df.groupby(group_cols)['rolling_temp'].transform(
    lambda x: x.replace(0, x[x != 0].mean() if not x[x != 0].empty else 0)
)

In [33]:
outliers = df[df['rolling_temp'] == 0]
outliers

# 이상치 처리 완료

,rolling_date,year,month,day,time,scale,spec_long,spec_long(1),spec_long(2),spec_long(3),...,fur_heat_time,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,항복강도(MPa),용도


# 파생변수 생성

In [ ]:
#유지-압연 온도 차이
df["soak_rolling_temp_diff"] = df["fur_soak_temp"] - df["rolling_temp"]

#전체 온도 감소율
df["total_temp_drop_rate"] = (df["fur_heat_temp"] - df["rolling_temp"]) / df["fur_heat_temp"] * 100

# 냉각 속도 지표
df["cooling_rate"] = (df["fur_soak_temp"] - df["rolling_temp"]) / (df["fur_total_time"] - df["fur_heat_time"] - df["fur_soak_time"])

# 열효율성 복합 지표
df["thermal_efficiency_index"] = (df["fur_heat_temp"] / df["fur_heat_time"]) * (df["fur_soak_temp"] / df["fur_soak_time"]) / (df["fur_heat_temp"] - df["rolling_temp"])

# 온도 안정성 지표
df["temp_stability"] = 1 / df[["fur_heat_temp", "fur_soak_temp", "rolling_temp"]].std(axis=1)

df.head()

# 데이터 분할

In [36]:
le = LabelEncoder()
df['scale_encoded'] = le.fit_transform(df['scale'])  # '불량'=0, '양품'=1
df.head()

,rolling_date,year,month,day,time,scale,spec_long,spec_long(1),spec_long(2),spec_long(3),...,fur_soak_temp,fur_soak_time,fur_total_time,rolling_method,rolling_temp,descaling_count,work_group,항복강도(MPa),용도,scale_encoded
0,07JAN2023:14:07:43,2023,JAN,07,14:07:43,불량,A131-DH36TM,A131,DH36TM,,...,1121,145,332,CR(제어압연),859.0,6,3조,355.0,선체구조용,0
1,08JAN2023:21:04:46,2023,JAN,08,21:04:46,양품,A283-C,A283,C,,...,1155,86,237,CR(제어압연),981.0,10,4조,245.0,용접구조용,1
2,08JAN2023:23:54:27,2023,JAN,08,23:54:27,불량,A283-C,A283,C,,...,1160,63,223,CR(제어압연),1026.0,8,4조,245.0,용접구조용,0
3,09JAN2023:00:04:11,2023,JAN,09,00:04:11,불량,A283-C,A283,C,,...,1177,65,230,CR(제어압연),1036.0,8,4조,245.0,용접구조용,0
4,09JAN2023:11:39:50,2023,JAN,09,11:39:50,불량,A283-C,A283,C,,...,1164,57,210,CR(제어압연),1034.0,8,3조,245.0,용접구조용,0


In [ ]:
# stratify 기준은 'scale_encoded' 컬럼 (불량'=0, '양품'=1)
df_train, df_test = train_test_split(
    df,
    test_size=0.3,
    random_state=1234,
    stratify=df['scale_encoded']  # 불량/양품 비율 똑같이
)

print("Train data size :", df_train.shape)
print("Test data size  :", df_test.shape)

# 클래스 비율 확인 (선택)
print("\n[Train] 클래스 분포:\n", df_train['scale_encoded'].value_counts(normalize=True))
print("\n[Test] 클래스 분포:\n", df_test['scale_encoded'].value_counts(normalize=True))

In [ ]:
# 데이터 분리:설명변수, 목표변수 구분
df_raw_x = df.drop("scale_encoded", axis = 1, inplace = False)
df_raw_y = df["scale_encoded"]

# 데이터 분할 train_test_split(X: 설명변수, Y: 목표변수, test_size = test 데이터 비율)
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(
    df_raw_x, df_raw_y, test_size = 0.3, random_state = 1234, stratify=df['scale_encoded'])

print("분할 전 설명변수 데이터 :", df_raw_x.shape)
print("분할 후 설명변수 데이터 :Train", df_train_x.shape, "  Test",df_test_x.shape)

**그리드 서치 범위 조정 필요**

# 1. 다중회귀분석

In [ ]:
# from_formula 함수를 이용하여 변수 역할 지정
log_model = Logit.from_formula("""scale_encoded ~ 변수들""", df_train)
# 적합
log_result = log_model.fit()

# 결과 출력
print(log_result.summary())

In [ ]:
# 회귀계수가 유의한 변수만 사용한 모델
log_model = Logit.from_formula("""scale_encoded ~ 변수들""", df_train)
# 적합
log_result = log_model.fit()

# 결과 출력
print(log_result.summary())

- 예측 및 평가

In [ ]:
# train 데이터 예측
y_pred_train = log_result.predict(df_train)
# 0과 1의 값을 가진 class로 변환
y_pred_train_class = (y_pred_train > 0.5).astype(int)  # 0.5 : “1/0” 판정 임계값(1 발생 확률) 변경 가능
print("Train 예측 결과 \n", y_pred_train_class.head(), "\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["scale_encoded"],y_pred_train_class)),"\n")

# test 데이터 예측
y_pred_test = log_result.predict(df_test)
# 0과 1의 값을 가진 class로 변환
y_pred_test_class = (y_pred_test > 0.5).astype(int)
print("Test 예측 결과 \n", y_pred_test_class.head(),"\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["scale_encoded"],y_pred_test_class)),"\n")

In [ ]:
# 실제 train 데이터와 예측 결과 비교
print("Train 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_train["scale_encoded"], y_pred_train_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["scale_encoded"],y_pred_train_class)),"\n")
print(classification_report(df_train["scale_encoded"], y_pred_train_class, digits=3))

# 실제 train 데이터와 예측 결과 비교
print("Test 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_test["scale_encoded"], y_pred_test_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["scale_encoded"],y_pred_test_class)),"\n")
print(classification_report(df_test["scale_encoded"], y_pred_test_class, digits=3))

In [ ]:
# 목표변수의 빈도 불균형 : f1 score로 모델 평가
print("Train 예측/분류 결과")
print(classification_report(df_test["scale_encoded"], y_pred_test_class, target_names=['불량', '양품'], digits=3))

print("Test 예측/분류 결과")
# 목표변수의 빈도 불균형 : f1 score로 모델 평가
print(classification_report(df_test["scale_encoded"], y_pred_test_class, target_names=['불량', '양품'], digits=3))

- 설명변수 중요도

In [ ]:
# 설명변수 중요도
df_logistic_coef = pd.DataFrame({"Coef": log_result.params.values[1:]}, index = log_model.exog_names[1:])
df_logistic_coef.plot.barh(y = "Coef", figsize=(10,6))

# 다중회귀분석(표준화 진행)

In [ ]:
# select_dtypes: 특정 변수 타입을 선택/제외하여 데이터 추출
df_char = df.select_dtypes(include = "object")
df_numeric = df.select_dtypes(exclude = "object")

# Data Scaling
scaler = StandardScaler()
np_numeric_scaled = scaler.fit_transform(df_numeric)
df_numeric_scaled = pd.DataFrame(np_numeric_scaled, columns = df_numeric.columns)

# 문자 데이터 + 숫자 데이터
df_scaled = pd.concat([df_numeric_scaled, df_char],axis = 1)
df_scaled.head()

In [ ]:
# 데이터를 0과 1로 변환(정수형), np.where(조건, 조건을 만족하는 경우, 만족하지 않는 경우)
df_scaled["scale_encoded"] = np.where(df_scaled["scale_encoded"]> 0, 1, 0)
df_scaled.head()

In [ ]:
# 데이터 분할
df_scaled_train, df_scaled_test = train_test_split(df_scaled, # 데이터
                               test_size = 0.3, # test 데이터의 비율
                               stratify=df['scale_encoded'],
                               random_state = 1234)  # random state

In [ ]:
# from_formula 함수를 이용하여 변수 역할 지정
# 선정된 설명변수 기준
scaled_log_model = Logit.from_formula("""scale_encoded ~ 변수들 """, df_scaled_train)
# 적합
scaled_log_result = scaled_log_model.fit()
# 결과 출력
print(scaled_log_result.summary())

- 예측 및 평가

In [ ]:
# train 데이터 예측
y_pred_train = log_result.predict(df_train)
# 0과 1의 값을 가진 class로 변환
y_pred_train_class = (y_pred_train > 0.5).astype(int)  # 0.5 : “1/0” 판정 임계값(1 발생 확률) 변경 가능
print("Train 예측 결과 \n", y_pred_train_class.head(), "\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["scale_encoded"],y_pred_train_class)),"\n")

# test 데이터 예측
y_pred_test = log_result.predict(df_test)
# 0과 1의 값을 가진 class로 변환
y_pred_test_class = (y_pred_test > 0.5).astype(int)
print("Test 예측 결과 \n", y_pred_test_class.head(),"\n")
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["scale_encoded"],y_pred_test_class)),"\n")

In [ ]:
# 실제 train 데이터와 예측 결과 비교
print("Train 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_train["scale_encoded"], y_pred_train_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_train["scale_encoded"],y_pred_train_class)),"\n")
print(classification_report(df_train["scale_encoded"], y_pred_train_class, digits=3))

# 실제 train 데이터와 예측 결과 비교
print("Test 예측/분류 결과")
print("Accuracy: {0:.3f}\n".format(accuracy_score(df_test["scale_encoded"], y_pred_test_class)))
print("Confusion Matrix: \n{}".format(confusion_matrix(df_test["scale_encoded"],y_pred_test_class)),"\n")
print(classification_report(df_test["scale_encoded"], y_pred_test_class, digits=3))

In [ ]:
# 설명변수 중요도
df_log_scaled_coef = pd.DataFrame({"Coef": scaled_log_result.params.values[1:]}, index = scaled_log_model.exog_names[1:])
df_log_scaled_coef.plot.barh(y = "Coef", legend = False)

# 2. Decision Tree

- 기본 모델

In [ ]:
tree_uncust = DecisionTreeClassifier(random_state=1234)
tree_uncust.fit(df_train_x, df_train_y)

# train 데이터 정확도
print("Accuracy on training set: {:.3f}".format(tree_uncust.score(df_train_x, df_train_y)))
# test 데이터 정확도
print("Accuracy on test set: {:.3f}".format(tree_uncust.score(df_test_x, df_test_y)))

- Grid Search

In [ ]:
estimator = DecisionTreeClassifier()

# 구하고자 하는 parameter와 범위
param_grid = {
    'max_depth': [3, 5, 7, 10, None],               # 트리 깊이
    'min_samples_split': [2, 5, 10],                # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1, 2, 4],                  # 리프 노드가 되기 위한 최소 샘플 수
    'criterion': ['gini', 'entropy']                # 불순도 지표
}

# 정확도가 높은 최적 parameter 찾기
grid_dt = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_dt.fit(df_train_x, df_train_y)

print("best estimator model: \n{}".format(grid_dt.best_estimator_))
print("\nbest parameter: \n{}".format(grid_dt.best_params_))
print("\nbest score: \n{}".format(grid_dt.best_score_.round(3)))

- 최종 모델

In [ ]:
tree_grid = DecisionTreeClassifier(max_depth=5, min_samples_split=15, min_samples_leaf=10, random_state=1234)
tree_grid.fit(df_train_x, df_train_y)

In [ ]:
# 예측
y_train_pred = tree_grid.predict(df_train_x)
y_test_pred = tree_grid.predict(df_test_x)

# 정확도
print("[Train] Accuracy: {:.3f}".format(accuracy_score(df_train_y, y_train_pred)))
print("[Test] Accuracy: {:.3f}".format(accuracy_score(df_test_y, y_test_pred)))

# 분류 리포트
print("\n[Test] Classification Report:\n")
print(classification_report(df_test_y, y_test_pred, target_names=['양품', '불량']))

# confusion matrix 시각화
cm = confusion_matrix(df_test_y, y_test_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['양품', '불량'], yticklabels=['양품', '불량'])
plt.title(' Confusion Matrix (Test Set)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

- 변수 중요도

In [ ]:
# 변수 중요도 추출
importances = tree_grid.feature_importances_
feature_names = df_train_x.columns

# 데이터프레임으로 정리
feature_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 상위 변수 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_df.head(15))
plt.title(' Feature Importance (Top 15)')
plt.tight_layout()
plt.show()

# 3. RandomForest

- 기본 모델

In [ ]:
# 기본 랜덤포레스트 모델 생성 및 학습
rf_uncust = RandomForestClassifier(random_state=1234)
rf_uncust.fit(df_train_x, df_train_y)

# 정확도 출력
print(" [기본 모델]")
print("Accuracy on training set: {:.3f}".format(rf_uncust.score(df_train_x, df_train_y)))
print("Accuracy on test set: {:.3f}".format(rf_uncust.score(df_test_x, df_test_y)))

- 그리드 서치

In [ ]:
# 파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# 모델 정의
rf_model = RandomForestClassifier(random_state=1234)

# GridSearchCV 정의
grid_rf = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 학습
grid_rf.fit(df_train_x, df_train_y)

# 결과 출력
print("\n [그리드 서치 결과]")
print("Best Parameters:", grid_rf.best_params_)
print("Best CV Accuracy: {:.3f}".format(grid_rf.best_score_))

- 예측 및 평가

In [ ]:
# 예측
best_rf = grid_rf.best_estimator_
y_test_pred = best_rf.predict(df_test_x)

# 정확도
print("\n Test Accuracy: {:.3f}".format(accuracy_score(df_test_y, y_test_pred)))

# 분류 리포트
print("\n Classification Report:")
print(classification_report(df_test_y, y_test_pred, target_names=['양품', '불량']))

# 혼동 행렬
cm = confusion_matrix(df_test_y, y_test_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', xticklabels=['양품', '불량'], yticklabels=['양품', '불량'])
plt.title(" Random Forest Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

- 변수 중요도

In [ ]:
# 변수 중요도 정리
importances = best_rf.feature_importances_
feature_df = pd.DataFrame({
    'Feature': df_train_x.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_df.head(15))
plt.title(" Random Forest Feature Importances (Top 15)")
plt.tight_layout()
plt.show()

# 4. Gradient Boosting

- 기본 모델

In [ ]:
# 그래디언트 부스팅 모델 생성: GradientBoostingClassifier
gb_uncust = GradientBoostingClassifier(random_state=1234)
gb_uncust.fit(df_train_x, df_train_y)

# train 모델 정확도
print("Accuracy on training set: {:.3f}".format(gb_uncust.score (df_train_x, df_train_y)))
# test 모델 정확도
print("Accuracy on test set: {:.3f}".format(gb_uncust.score (df_test_x, df_test_y)))

- 그리드 서치

In [ ]:
estimator = GradientBoostingClassifier(random_state=1234)

param_grid = {
    'n_estimators': [30, 45, 60, 80],             # 트리 개수
    'learning_rate': [0.01, 0.05, 0.1, 0.2],      # 학습률
    'max_depth': [3, 5, 7, 10],                   # 트리 깊이
    'min_samples_split': [2, 5, 10],              # 노드 분할 최소 샘플 수
    'min_samples_leaf': [1, 2, 4]                 # 리프 노드 최소 샘플 수
}

# GridSearchCV 정의
grid_gb = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 학습
grid_gb.fit(df_train_x, df_train_y)

# 결과 출력
print("\n best estimator model: \n", grid_gb.best_estimator_)
print("\n best parameters: \n", grid_gb.best_params_)
print("\n best cross-validation score: {:.3f}".format(grid_gb.best_score_))

- 예측 및 평가

In [ ]:
# 테스트 데이터 예측
y_test_pred = grid_gb.best_estimator_.predict(df_test_x)

# 분류 리포트 출력
print("📄 Classification Report (Test Data):\n")
print(classification_report(df_test_y, y_test_pred, target_names=['양품', '불량']))

# Confusion Matrix 시각화
cm = confusion_matrix(df_test_y, y_test_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['양품', '불량'], yticklabels=['양품', '불량'])
plt.title('Confusion Matrix (Test Set)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

- 변수 중요도

In [ ]:
# 변수 중요도 가져오기
importances = grid_gb.best_estimator_.feature_importances_
feature_names = df_train_x.columns

# 중요도 데이터프레임 생성
feature_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_df.head(15))
plt.title('Top 15 Feature Importances (Gradient Boosting)')
plt.tight_layout()
plt.show()

# 5. XGBoost

- 기본 모델

In [ ]:
# 기본 XGBoost 모델 생성
xgb_uncust = XGBClassifier(random_state=1234, use_label_encoder=False, eval_metric='logloss')
xgb_uncust.fit(df_train_x, df_train_y)

# 정확도 출력
print("Accuracy on training set: {:.3f}".format(xgb_uncust.score(df_train_x, df_train_y)))
print("Accuracy on test set: {:.3f}".format(xgb_uncust.score(df_test_x, df_test_y)))

- 그리드 서치

In [ ]:
# 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# XGBoost 분류기 객체
xgb_model = XGBClassifier(random_state=1234, use_label_encoder=False, eval_metric='logloss')

# 그리드 서치 정의
grid_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 모델 학습
grid_xgb.fit(df_train_x, df_train_y)

# 결과 출력
print("\n [그리드 서치 결과]")
print("Best Parameters:", grid_xgb.best_params_)
print("Best CV Accuracy: {:.3f}".format(grid_xgb.best_score_))

- 예측 및 평가

In [ ]:
# 최적 모델로 예측
best_xgb = grid_xgb.best_estimator_
y_test_pred = best_xgb.predict(df_test_x)

# 정확도
print("\n Test Set Accuracy: {:.3f}".format(accuracy_score(df_test_y, y_test_pred)))

# 분류 리포트
print("\n Classification Report:")
print(classification_report(df_test_y, y_test_pred, target_names=['양품', '불량']))

# 혼동 행렬
cm = confusion_matrix(df_test_y, y_test_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['양품', '불량'], yticklabels=['양품', '불량'])
plt.title(" Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

- 변수 중요도

In [ ]:
# 변수 중요도 추출
importances = best_xgb.feature_importances_
feature_df = pd.DataFrame({
    'Feature': df_train_x.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_df.head(15))
plt.title(" XGBoost Feature Importances (Top 15)")
plt.tight_layout()
plt.show()

# 6. lightGBM

- 기본 모델

In [ ]:
# 기본 LGBM 모델 생성 및 학습
lgbm_uncust = LGBMClassifier(random_state=1234)
lgbm_uncust.fit(df_train_x, df_train_y)

# 정확도 출력
print(" [기본 모델]")
print("Accuracy on training set: {:.3f}".format(lgbm_uncust.score(df_train_x, df_train_y)))
print("Accuracy on test set: {:.3f}".format(lgbm_uncust.score(df_test_x, df_test_y)))

- 그리드 서치

In [ ]:
# 파라미터 범위 지정
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [-1, 5, 10],
    'num_leaves': [15, 31, 63],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# GridSearchCV 실행
lgbm_model = LGBMClassifier(random_state=1234)
grid_lgbm = GridSearchCV(
    estimator=lgbm_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 학습
grid_lgbm.fit(df_train_x, df_train_y)

# 결과 출력
print("\n [그리드 서치 결과]")
print("Best Parameters:", grid_lgbm.best_params_)
print("Best CV Accuracy: {:.3f}".format(grid_lgbm.best_score_))

- 예측 및 평가

In [ ]:
# 최적 모델 예측
best_lgbm = grid_lgbm.best_estimator_
y_test_pred = best_lgbm.predict(df_test_x)

# 정확도
print("\n Test Accuracy: {:.3f}".format(accuracy_score(df_test_y, y_test_pred)))

# 분류 리포트
print("\n Classification Report:")
print(classification_report(df_test_y, y_test_pred, target_names=['양품', '불량']))

# 혼동 행렬
cm = confusion_matrix(df_test_y, y_test_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Purples', xticklabels=['양품', '불량'], yticklabels=['양품', '불량'])
plt.title(" LightGBM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

- 변수 중요도

In [ ]:
# 변수 중요도 정리
importances = best_lgbm.feature_importances_
feature_df = pd.DataFrame({
    'Feature': df_train_x.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_df.head(15))
plt.title(" LightGBM Feature Importances (Top 15)")
plt.tight_layout()
plt.show()

# 7. SVM

- 기본 모델

In [ ]:
# 기본 SVM 모델 생성 (선형 커널이 기본)
svm_uncust = SVC(random_state=1234)
svm_uncust.fit(df_train_x, df_train_y)

# 정확도 출력
print(" [기본 모델]")
print("Accuracy on training set: {:.3f}".format(svm_uncust.score(df_train_x, df_train_y)))
print("Accuracy on test set: {:.3f}".format(svm_uncust.score(df_test_x, df_test_y)))

- 그리드 서치

In [ ]:
# 파라미터 그리드 설정 (커널별로 분기 가능)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto']  # rbf, poly용
}

svm_model = SVC(random_state=1234)

# GridSearchCV 정의
grid_svm = GridSearchCV(
    estimator=svm_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 학습
grid_svm.fit(df_train_x, df_train_y)

# 결과 출력
print("\n [그리드 서치 결과]")
print("Best Parameters:", grid_svm.best_params_)
print("Best CV Accuracy: {:.3f}".format(grid_svm.best_score_))

- 예측 및 평가

In [ ]:
# 예측
best_svm = grid_svm.best_estimator_
y_test_pred = best_svm.predict(df_test_x)

# 정확도
print("\n Test Accuracy: {:.3f}".format(accuracy_score(df_test_y, y_test_pred)))

# 분류 리포트
print("\n Classification Report:")
print(classification_report(df_test_y, y_test_pred, target_names=['양품', '불량']))

# 혼동 행렬
cm = confusion_matrix(df_test_y, y_test_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=['양품', '불량'], yticklabels=['양품', '불량'])
plt.title(" SVM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()


- 변수 중요도

In [ ]:
# 선형 커널인 경우만 coef_ 사용 가능
if grid_svm.best_params_['kernel'] == 'linear':
    coef = best_svm.coef_[0]
    feature_df = pd.DataFrame({
        'Feature': df_train_x.columns,
        'Coefficient': coef,
        'Abs_Coefficient': np.abs(coef)
    }).sort_values(by='Abs_Coefficient', ascending=False)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Abs_Coefficient', y='Feature', data=feature_df.head(15))
    plt.title(" SVM (Linear) Feature Importance (Top 15)")
    plt.tight_layout()
    plt.show()
else:
    print(" 변수 중요도는 선형 커널에서만 시각화할 수 있습니다.")

#  8. MLP

- 기본 모델

In [ ]:
# 기본 MLP 모델 생성
mlp_uncust = MLPClassifier(random_state=1234, max_iter=500)  # 기본 max_iter 늘려야 수렴
mlp_uncust.fit(df_train_x, df_train_y)

# 정확도 출력
print(" [기본 MLP 모델]")
print("Accuracy on training set: {:.3f}".format(mlp_uncust.score(df_train_x, df_train_y)))
print("Accuracy on test set: {:.3f}".format(mlp_uncust.score(df_test_x, df_test_y)))

- 그리드 서치

In [ ]:
# 하이퍼파라미터 그리드
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (100, 100)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],  # L2 정규화
    'learning_rate': ['constant', 'adaptive']
}

# 모델 정의
mlp_model = MLPClassifier(random_state=1234, max_iter=500)

# GridSearchCV 설정
grid_mlp = GridSearchCV(
    estimator=mlp_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    n_jobs=-1,
    verbose=1
)

# 모델 학습
grid_mlp.fit(df_train_x, df_train_y)

# 결과 출력
print("\n [그리드 서치 결과]")
print("Best Parameters:", grid_mlp.best_params_)
print("Best CV Accuracy: {:.3f}".format(grid_mlp.best_score_))

- 예측 및 평가

In [ ]:
# 최적 모델로 예측
best_mlp = grid_mlp.best_estimator_
y_test_pred = best_mlp.predict(df_test_x)

# 정확도 출력
print("\n Test Accuracy: {:.3f}".format(accuracy_score(df_test_y, y_test_pred)))

# 분류 리포트
print("\n Classification Report:")
print(classification_report(df_test_y, y_test_pred, target_names=['양품', '불량']))

# 혼동행렬 시각화
cm = confusion_matrix(df_test_y, y_test_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges', xticklabels=['양품', '불량'], yticklabels=['양품', '불량'])
plt.title(" MLP Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()

- 변수 중요도

In [ ]:
# permutation importance 계산
perm_result = permutation_importance(best_mlp, df_test_x, df_test_y, n_repeats=10, random_state=1234)

# 정리
feature_df = pd.DataFrame({
    'Feature': df_test_x.columns,
    'Importance': perm_result.importances_mean
}).sort_values(by='Importance', ascending=False)

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=feature_df.head(15))
plt.title(" MLP Permutation Feature Importances (Top 15)")
plt.tight_layout()
plt.show()